# LangChain 1.0 & LangGraph 1.0 升级教程

**面向人群**：多智能体系统中高级开发者  
**升级路径**：LangChain 0.3.x → 1.0 / LangGraph 0.6.x → 1.0  
**参考文档**：[LangChain 官方文档](https://docs.langchain.com)

---

## 📋 教程结构

1. **环境准备** - 安装与配置
2. **核心概念** - 架构对比与变更
3. **LangChain 1.0** - create_agent、Middleware、Content Blocks
4. **LangGraph 1.0** - StateGraph、Checkpointer 持久化
5. **完整案例** - 端到端迁移示例
6. **最佳实践** - 调试技巧与性能优化

---
## 第 1 部分：环境准备

### 1.1 检查 Python 版本

In [ ]:
import sys

print(f"Python 版本: {sys.version}")

if sys.version_info < (3, 10):
    print("⚠️ 警告: LangChain 1.0 需要 Python 3.10+")
else:
    print("✅ Python 版本符合要求")

### 1.2 安装依赖

In [ ]:
!pip install -qU langchain langgraph langchain-core langchain-openai langchain-anthropic

### 1.3 验证安装

In [ ]:
import langchain
import langgraph

print(f"LangChain: {langchain.__version__}")
print(f"LangGraph: {langgraph.__version__}")

from langchain.agents import create_agent
print("✅ 核心模块导入成功")

### 1.4 配置 API Keys

In [ ]:
import os
from getpass import getpass

# 根据使用的模型配置 API Key
# os.environ["OPENAI_API_KEY"] = getpass("OpenAI API Key: ")
# os.environ["ANTHROPIC_API_KEY"] = getpass("Anthropic API Key: ")

print("✅ 配置完成")

---
## 第 2 部分：核心概念对比

### 2.1 架构关系

```
┌─────────────────────────────┐
│  LangChain 1.0 (高层框架)   │
│  • create_agent            │
│  • Middleware              │
│  • Content Blocks          │
└──────────┬──────────────────┘
           │ 运行在
           ↓
┌─────────────────────────────┐
│  LangGraph 1.0 (运行时)     │
│  • StateGraph              │
│  • Checkpointer            │
│  • HITL & Streaming        │
└─────────────────────────────┘
```

### 2.2 关键变更速查

In [ ]:
import pandas as pd

changes = pd.DataFrame({
    "功能": [
        "智能体创建",
        "Prompt",
        "Hooks",
        "状态类型",
        "Context"
    ],
    "v0.x": [
        "create_react_agent",
        "prompt=SystemMessage",
        "state_modifier",
        "TypedDict/Pydantic",
        "config['configurable']"
    ],
    "v1.0": [
        "create_agent",
        "system_prompt=str",
        "Middleware",
        "仅 TypedDict",
        "context 参数"
    ]
})

display(changes)

---
## 第 3 部分：LangChain 1.0 实战

### 3.1 创建基础智能体

In [ ]:
from langchain.agents import create_agent
from langchain.tools import tool

@tool
def get_weather(city: str) -> str:
    """获取城市天气"""
    return f"{city}: 晴天 20°C"

@tool
def calculate(expr: str) -> str:
    """计算数学表达式"""
    try:
        return f"结果: {eval(expr)}"
    except:
        return "计算错误"

agent = create_agent(
    model="gpt-4o-mini",
    tools=[get_weather, calculate],
    system_prompt="你是一个助手"
)

print("✅ 智能体创建成功")

In [ ]:
# 调用智能体
result = agent.invoke({
    "messages": [{"role": "user", "content": "北京天气?"}]
})

print(result["messages"][-1].content)

### 3.2 Middleware 示例

#### 内置 PII 脱敏

In [ ]:
from langchain.agents.middleware import PIIMiddleware

pii_agent = create_agent(
    model="gpt-4o-mini",
    tools=[get_weather],
    middleware=[
        PIIMiddleware("email", strategy="redact")
    ]
)

result = pii_agent.invoke({
    "messages": [{"role": "user", "content": "user@example.com"}]
})

print("PII 脱敏:", result["messages"][-1].content)

#### 自定义 Middleware

In [ ]:
from langchain.agents.middleware import AgentMiddleware

class LogMiddleware(AgentMiddleware):
    def before_model(self, request):
        print(f"📝 调用 LLM: {len(request.state['messages'])} 条消息")
        return None
    
    def after_model(self, state, runtime):
        print(f"✅ LLM 响应完成")
        return None

log_agent = create_agent(
    model="gpt-4o-mini",
    tools=[get_weather],
    middleware=[LogMiddleware()]
)

result = log_agent.invoke({
    "messages": [{"role": "user", "content": "天气?"}]
})

### 3.3 Content Blocks

In [ ]:
from langchain.chat_models import init_chat_model

model = init_chat_model("gpt-4o-mini")
response = model.invoke("什么是 AI?")

print(f"Blocks 数量: {len(response.content_blocks)}")
for block in response.content_blocks:
    if block["type"] == "text":
        print(f"文本: {block['text'][:50]}...")

### 3.4 结构化输出

In [ ]:
from langchain.agents.structured_output import ToolStrategy
from pydantic import BaseModel

class Weather(BaseModel):
    city: str
    temp: float
    condition: str

struct_agent = create_agent(
    model="gpt-4o-mini",
    tools=[get_weather],
    response_format=ToolStrategy(Weather)
)

result = struct_agent.invoke({
    "messages": [{"role": "user", "content": "北京天气"}]
})

weather = result["structured_response"]
print(f"城市: {weather.city}")
print(f"温度: {weather.temp}°C")

---
## 第 4 部分：LangGraph 1.0 实战

### 4.1 StateGraph 基础

In [ ]:
from langgraph.graph import StateGraph, END
from typing import TypedDict

class State(TypedDict):
    count: int
    messages: list

def increment(state: State) -> State:
    print(f"🔵 Count: {state['count']}")
    return {"count": state["count"] + 1}

def router(state: State) -> str:
    return "end" if state["count"] >= 3 else "continue"

graph = StateGraph(State)
graph.add_node("inc", increment)
graph.add_conditional_edges("inc", router, {
    "continue": "inc",
    "end": END
})
graph.set_entry_point("inc")

app = graph.compile()
print("✅ Graph 构建完成")

In [ ]:
result = app.invoke({"count": 0, "messages": []})
print(f"\n最终 count: {result['count']}")

### 4.2 Checkpointer 持久化

In [ ]:
from langgraph.checkpoint.memory import MemorySaver

# 使用内存 Checkpointer
memory = MemorySaver()
app_with_memory = graph.compile(checkpointer=memory)

config = {"configurable": {"thread_id": "thread-1"}}

# 第一次调用
r1 = app_with_memory.invoke(
    {"count": 0, "messages": []}, 
    config
)
print(f"第 1 次: count = {r1['count']}")

# 第二次调用（从检查点恢复）
r2 = app_with_memory.invoke(
    {"count": 0, "messages": []},  # count 从检查点恢复
    config
)
print(f"第 2 次: count = {r2['count']} (从检查点继续)")

### 4.3 SQLite 持久化

In [ ]:
from langgraph.checkpoint.sqlite import SqliteSaver

# SQLite 持久化
sqlite = SqliteSaver.from_conn_string(":memory:")
app_sqlite = graph.compile(checkpointer=sqlite)

result = app_sqlite.invoke(
    {"count": 0, "messages": []},
    {"configurable": {"thread_id": "sql-thread"}}
)

print(f"✅ SQLite 持久化: count = {result['count']}")

---
## 第 5 部分：完整迁移案例

### 5.1 v0.x 旧代码

In [ ]:
# 旧版代码示例（已弃用，仅作演示）
OLD_CODE = """
from langgraph.prebuilt import create_react_agent
from langchain.messages import SystemMessage

agent = create_react_agent(
    model,
    tools,
    prompt=SystemMessage(content="You are helpful"),
    state_modifier=custom_modifier
)
"""

print("❌ 旧版代码 (v0.x):")
print(OLD_CODE)

### 5.2 v1.0 新代码

In [ ]:
from langchain.agents import create_agent
from langchain.agents.middleware import AgentMiddleware
from langchain.tools import tool

@tool
def search(query: str) -> str:
    """搜索工具"""
    return f"搜索结果: {query}"

class CustomModifier(AgentMiddleware):
    def before_model(self, request):
        # 自定义逻辑
        return None

# ✅ 新版代码 (v1.0)
new_agent = create_agent(
    model="gpt-4o-mini",
    tools=[search],
    system_prompt="You are helpful",
    middleware=[CustomModifier()]
)

print("✅ 新版代码已创建")

In [ ]:
# 测试新版智能体
result = new_agent.invoke({
    "messages": [{"role": "user", "content": "搜索 AI"}]
})

print("🤖 响应:", result["messages"][-1].content)

---
## 第 6 部分：最佳实践

### 6.1 调试技巧

In [ ]:
# 启用详细日志
import logging
logging.basicConfig(level=logging.INFO)

# 流式调试
for event in agent.stream({
    "messages": [{"role": "user", "content": "测试"}]
}):
    node = list(event.keys())[0]
    print(f"[{node}] 执行完成")

### 6.2 性能优化

In [ ]:
from langchain.agents.middleware import SummarizationMiddleware

# 对话历史自动摘要
optimized_agent = create_agent(
    model="gpt-4o-mini",
    tools=[search],
    middleware=[
        SummarizationMiddleware(
            model="gpt-4o-mini",
            trigger={"tokens": 1000}
        )
    ]
)

print("✅ 优化配置完成")

### 6.3 迁移检查清单

In [ ]:
checklist = pd.DataFrame({
    "步骤": [
        "1. Python >= 3.10",
        "2. 升级依赖包",
        "3. 更新导入路径",
        "4. create_agent",
        "5. 迁移 Middleware",
        "6. TypedDict State",
        "7. 测试验证"
    ],
    "状态": ["☐"] * 7
})

display(checklist)

---
## 📚 参考资源

### 官方文档

- [LangChain 1.0 Release](https://docs.langchain.com/oss/python/releases/langchain-v1)
- [LangGraph 1.0 Release](https://docs.langchain.com/oss/python/releases/langgraph-v1)
- [Migration Guide](https://docs.langchain.com/oss/python/migrate/langchain-v1)
- [API Reference](https://reference.langchain.com/python/)

### 社区

- [GitHub Issues](https://github.com/langchain-ai/langchain/issues?q=label%3Av1)
- [Discord](https://discord.gg/langchain)
- [Forum](https://forum.langchain.com/)

---
## 🎯 总结

### 核心要点

1. **LangChain 1.0**：从工具包到生产级框架
   - ✅ 统一 `create_agent` 接口
   - ✅ 可组合 Middleware 系统
   - ✅ 跨供应商 Content Blocks

2. **LangGraph 1.0**：稳定的运行时
   - ✅ StateGraph API 不变
   - ✅ 生产级持久化
   - ✅ 深度集成 LangChain

3. **升级策略**
   - ✅ 新项目直接用 v1.0
   - ✅ 现有项目渐进迁移
   - ✅ 使用 langchain-classic 过渡

### 下一步

- 🔨 实践：基于案例修改自己的项目
- 📖 深入：阅读官方文档进阶特性
- 💬 交流：加入社区分享经验